In [ ]:
# default_exp uniformis

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Uniform IS

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_pivot_selected_folder  = "D:/data/parq_pivot_select"
all_pivoted_folder = "D:/data/parq_pivot_split"
all_processed_folder = "D:/data/parq_processed/"

col_list =    ["stmt","cik","ticker", "adsh","period","filed", "form","tag","value","report", "line", "fp", "uom"]
pivot_group = ["cik","ticker","adsh","form","period","filed","fp", "qtrs"]

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 00_Raw_data

In [ ]:
# loading the complete unpivoted dataset - if it is needed for debbuging
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# it happens sometimes, that the data could not be associated with a right sheet (bs, is, cf, ..). in this cases, the data can appea under "UN"
# so if expected information cannot be found in the appropriate statement, we have to look in the un statement
un_pivot_value = load_data(all_pivoted_folder, spark, "UN", "value")
un_pivot_pd = un_pivot_value.toPandas()

In [ ]:
un_pivot_pd.shape

(5989, 1775)

In [ ]:
def prepare_un_values(df_to_merge_into, attr_list):
    # add possible columns from un set to  cf data with prefix cpy_
    attributes = pivot_group[:] # create copy
    attributes.extend(attr_list)

    un_prepared = un_pivot_pd[attributes].copy()
    un_prepared.rename(columns=lambda x: x  if x in pivot_group else ("cpy_" + x), inplace=True)

    return pd.merge(df_to_merge_into, un_prepared, how='left', on=pivot_group)

## 01_IncomeStatement

Gross Margin
- Net Sales
- Cost of Sales
- Gross Margin -> NetSales - CostOfSales

Operating Expenses
- R&D
- Selling, general and admin
- Total op expenses = R&D + Selling, general and admin

- Operating Income = Gross Margin - Total op expenses      -> OperatingIncomeLoss
- other income
- Income before provision for income taxes = operating income + other income

- Provision for income taxes
- Net income = Income before taxes -taxes                   -> NetIncomeLoss -> also available in CF(!)

Earning per share
- Basic
- Diluted

Shares used in computing earnings per share:
- basic
- diluted


In [ ]:
is_pivot_value = load_data(all_pivoted_folder, spark, "IS", "value")
spark_shape(is_pivot_value)

(116298, 1928)

In [ ]:
is_pivot_pd = is_pivot_value.toPandas()

In [ ]:
is_pivot_pd['value_count'] = is_pivot_pd.notnull().sum(axis=1)-len(pivot_group) # create a column that countains the number of not null values of the row
# if there are less than 5 columns with values it is likely that this is not a complete statement
# often this indicates, that the real information is inside the ComprehensiveIncome Statement and not in an IncomeStatement
is_pivot_pd = is_pivot_pd[is_pivot_pd['value_count'] > 4] 

In [ ]:
is_pivot_pd_copy = is_pivot_pd.copy()

In [ ]:
is_pivot_pd_copy.shape

(115948, 1929)

### Merge Data from "unknown" statement

In [ ]:
# merge relevant columns from the UN dataset
is_pivot_pd_copy = prepare_un_values(is_pivot_pd_copy, [
    'EarningsPerShareBasic',
    'EarningsPerShareBasicAndDiluted',
    'EarningsPerShareDiluted',
    #'EarningsPerShareBasicDistributed',   
    #'EarningsPerShareDilutedDistributed',
    
    'WeightedAverageNumberOfSharesOutstandingBasic',
    'WeightedAverageNumberOfDilutedSharesOutstanding',
    
    'NetIncomeLoss',
    'NetIncomeLossAvailableToCommonStockholdersBasic',
    'NetIncomeLossAllocatedToLimitedPartners',
    'ProfitLoss',
    
    'Revenues',
    'SalesRevenueNet',
    'RevenueFromContractWithCustomerExcludingAssessedTax', 
    'RevenueFromContractWithCustomerIncludingAssessedTax', 
    'CostOfGoodsAndServicesSold',
    'CostOfGoodsSold',
    'CostOfRevenue',
    'CostOfServices',
    'CostsAndExpenses',
    'OperatingIncomeLoss',
    'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments',
    'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
    
    'GrossProfit',
])
is_pivot_pd_copy.shape

(115948, 1951)

### Shares

In [ ]:
print_null_count(is_pivot_pd_copy, [ 
 'EarningsPerShareBasic',
 'EarningsPerShareBasicAndDiluted',
 'EarningsPerShareDiluted',
 'EarningsPerShareBasicDistributed',   
 'EarningsPerShareDilutedDistributed',
])

EarningsPerShareBasic   28747
EarningsPerShareBasicAndDiluted   94600
EarningsPerShareDiluted   29583
EarningsPerShareBasicDistributed   115931
EarningsPerShareDilutedDistributed   115931


In [ ]:
is_pivot_pd_copy['EarningsPerShare_hj'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_EarningsPerShareBasic', 'EarningsPerShareBasic')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_EarningsPerShareBasicAndDiluted', 'EarningsPerShareBasicAndDiluted')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_EarningsPerShareDiluted', 'EarningsPerShareDiluted',)

copy_if_not_empty(is_pivot_pd_copy, 'EarningsPerShareBasic',              'EarningsPerShare_hj')
copy_if_not_empty(is_pivot_pd_copy, 'EarningsPerShareBasicAndDiluted',    'EarningsPerShare_hj')
copy_if_not_empty(is_pivot_pd_copy, 'EarningsPerShareBasicDistributed',   'EarningsPerShare_hj')
copy_if_not_empty(is_pivot_pd_copy, 'EarningsPerShareDiluted',            'EarningsPerShare_hj') 
copy_if_not_empty(is_pivot_pd_copy, 'EarningsPerShareDilutedDistributed', 'EarningsPerShare_hj') 

In [ ]:
print_null_count(is_pivot_pd_copy, ['EarningsPerShare_hj'])

EarningsPerShare_hj   7118


In [ ]:
print_null_count(is_pivot_pd_copy, ['WeightedAverageNumberOfSharesOutstandingBasic','WeightedAverageNumberOfDilutedSharesOutstanding'])

WeightedAverageNumberOfSharesOutstandingBasic   42196
WeightedAverageNumberOfDilutedSharesOutstanding   42425


In [ ]:
is_pivot_pd_copy['SharesOutstanding_hj'] = None

copy_if_not_empty(is_pivot_pd_copy, 'WeightedAverageNumberOfSharesOutstandingBasic', 'WeightedAverageNumberOfSharesOutstandingBasic')
copy_if_not_empty(is_pivot_pd_copy, 'WeightedAverageNumberOfDilutedSharesOutstanding', 'WeightedAverageNumberOfDilutedSharesOutstanding')

copy_if_not_empty(is_pivot_pd_copy, 'WeightedAverageNumberOfSharesOutstandingBasic', 'SharesOutstanding_hj')
copy_if_not_empty(is_pivot_pd_copy, 'WeightedAverageNumberOfDilutedSharesOutstanding', 'SharesOutstanding_hj')

In [ ]:
print_null_count(is_pivot_pd_copy, ['SharesOutstanding_hj'])

SharesOutstanding_hj   40992


### NetIncome

In [ ]:
print_null_count(is_pivot_pd_copy, [ 'NetIncomeLoss', 'NetIncomeLossAvailableToCommonStockholdersBasic', 'ProfitLoss'])

NetIncomeLoss   15061
NetIncomeLossAvailableToCommonStockholdersBasic   91942
ProfitLoss   76502


In [ ]:
is_pivot_pd_copy['NetIncomeLoss_hj'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_NetIncomeLoss', 'NetIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_NetIncomeLossAvailableToCommonStockholdersBasic', 'NetIncomeLossAvailableToCommonStockholdersBasic')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_NetIncomeLossAllocatedToLimitedPartners', 'NetIncomeLossAllocatedToLimitedPartners')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_ProfitLoss', 'ProfitLoss')


copy_if_not_empty(is_pivot_pd_copy, 'NetIncomeLoss', 'NetIncomeLoss_hj')
copy_if_not_empty(is_pivot_pd_copy, 'NetIncomeLossAvailableToCommonStockholdersBasic', 'NetIncomeLoss_hj')
copy_if_not_empty(is_pivot_pd_copy, 'NetIncomeLossAllocatedToLimitedPartners', 'NetIncomeLoss_hj')
copy_if_not_empty(is_pivot_pd_copy, 'ProfitLoss', 'NetIncomeLoss_hj')

In [ ]:
print_null_count(is_pivot_pd_copy, [ 'NetIncomeLoss_hj', 'NetIncomeLoss', 'ProfitLoss'])

NetIncomeLoss_hj   629
NetIncomeLoss   15040
ProfitLoss   76479


### NetSales / Revenues

In [ ]:
print_null_count(is_pivot_pd_copy, [ 
    'Revenues',
    'SalesRevenueNet',
    'RevenueFromContractWithCustomerExcludingAssessedTax', # Sales
    'RevenueFromContractWithCustomerIncludingAssessedTax', # Sales
])

Revenues   72902
SalesRevenueNet   85755
RevenueFromContractWithCustomerExcludingAssessedTax   105867
RevenueFromContractWithCustomerIncludingAssessedTax   112745


In [ ]:
is_pivot_pd_copy['Revenues_hj'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_Revenues', 'Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_SalesRevenueNet', 'SalesRevenueNet')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_RevenueFromContractWithCustomerExcludingAssessedTax', 'RevenueFromContractWithCustomerExcludingAssessedTax')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_RevenueFromContractWithCustomerIncludingAssessedTax', 'RevenueFromContractWithCustomerIncludingAssessedTax')
#copy_if_not_empty(is_pivot_pd_copy, 'cpy_RevenuesExcludingInterestAndDividends', 'RevenuesExcludingInterestAndDividends')

copy_if_not_empty(is_pivot_pd_copy, 'Revenues', 'Revenues_hj')
copy_if_not_empty(is_pivot_pd_copy, 'SalesRevenueNet', 'Revenues_hj')
copy_if_not_empty(is_pivot_pd_copy, 'RevenueFromContractWithCustomerExcludingAssessedTax', 'Revenues_hj')
copy_if_not_empty(is_pivot_pd_copy, 'RevenueFromContractWithCustomerIncludingAssessedTax', 'Revenues_hj')
copy_if_not_empty(is_pivot_pd_copy, 'RevenuesExcludingInterestAndDividends', 'Revenues_hj')
copy_if_not_empty(is_pivot_pd_copy, 'RegulatedAndUnregulatedOperatingRevenue', 'Revenues_hj')

In [ ]:
# some companies provide NonInterestIncome and InterestAndDividendIncomeOperating instead of a Revenue
sum_into_empty_target(is_pivot_pd_copy,  
                      'InterestAndDividendIncomeOperating',
                      'NoninterestIncome',
                      'Revenues_hj')

sum_into_empty_target(is_pivot_pd_copy,  
                      'InterestIncomeExpenseNet',
                      'NoninterestIncome',
                      'Revenues_hj')

In [ ]:
print_null_count(is_pivot_pd_copy, [ 'Revenues_hj'])

Revenues_hj   19707


### CostOfSales

In [ ]:
print_null_count(is_pivot_pd_copy, [ 
    'CostOfGoodsAndServicesSold',
    'CostOfGoodsSold',
    'CostOfRevenue',
    'CostOfServices',
])

CostOfGoodsAndServicesSold   92883
CostOfGoodsSold   92137
CostOfRevenue   97325
CostOfServices   106637


In [ ]:
is_pivot_pd_copy['CostOfRevenue_hj'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfGoodsAndServicesSold', 'CostOfGoodsAndServicesSold')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfGoodsSold', 'CostOfGoodsSold')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfRevenue', 'CostOfRevenue')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfServices', 'CostOfServices')

copy_if_not_empty(is_pivot_pd_copy, 'CostOfRevenue', 'CostOfRevenue_hj')
copy_if_not_empty(is_pivot_pd_copy, 'CostOfGoodsAndServicesSold', 'CostOfRevenue_hj')

sum_into_empty_target(is_pivot_pd_copy,  
                      'CostOfGoodsSold',
                      'CostOfServices',
                      'CostOfRevenue_hj')

copy_if_not_empty(is_pivot_pd_copy, 'CostOfGoodsSold', 'CostOfRevenue_hj')
copy_if_not_empty(is_pivot_pd_copy, 'CostOfServices', 'CostOfRevenue_hj')

In [ ]:
print_null_count(is_pivot_pd_copy, ['CostOfRevenue_hj'])

CostOfRevenue_hj   52428


### OperatingIncomeLoss

In [ ]:
print_null_count(is_pivot_pd_copy, ['OperatingIncomeLoss',
                                   'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments',
                                   'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest'])

OperatingIncomeLoss   28969
IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments   59865
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest   70481


In [ ]:
is_pivot_pd_copy['OperatingIncomeLoss_hj'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_OperatingIncomeLoss', 'OperatingIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest')

copy_if_not_empty(is_pivot_pd_copy, 'OperatingIncomeLoss', 'OperatingIncomeLoss_hj')
copy_if_not_empty(is_pivot_pd_copy, 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments', 'OperatingIncomeLoss_hj')
copy_if_not_empty(is_pivot_pd_copy, 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'OperatingIncomeLoss_hj')

In [ ]:
print_null_count(is_pivot_pd_copy, ['OperatingIncomeLoss_hj'])

OperatingIncomeLoss_hj   4110


### Other

In [ ]:
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostsAndExpenses', 'CostsAndExpenses')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_GrossProfit', 'GrossProfit')

#### Save

In [ ]:
is_pivot_pd_copy[["cik","ticker", "adsh","period","form", "qtrs","fp",
                    'Revenues_hj',
                    'CostOfRevenue_hj',
                    'GrossProfit',
                    'OperatingIncomeLoss_hj',
                    'CostsAndExpenses',
                    'NetIncomeLoss_hj', 'NetIncomeLoss', 'ProfitLoss',
                    'SharesOutstanding_hj',
                    'EarningsPerShare_hj'
                 ]] \
                  .to_csv(all_processed_folder + "is_not_cleaned.csv", index=False)

## XX_Trials